In [1]:
import pandas as pd
pd.set_option('display.max_rows',500)
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
plt.style.use("fivethirtyeight")
%matplotlib inline

# For reading stock data from yahoo
import pandas_datareader.data as pdr
from pandas_datareader.data import DataReader
import yfinance as yf

# For time stamps
import datetime

from prophet import Prophet

C:\Users\0074fasaet\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [2]:
# download stocks
file = pd.read_csv('M6_Universe.csv')
stocks_list = file['symbol'].values.tolist()
#stocks_list.remove('DRE')

end = datetime.datetime.now()
start = datetime.datetime(end.year - 3, end.month, end.day)

last_month_days = []
for year in range(start.year, end.year+1):
    for month in range(1,13):
        first_day = datetime.datetime(year, month, 1)
        prev_day = first_day - datetime.timedelta(days=1)
        last_month_days += [prev_day.strftime('%Y-%m-%d')]

In [3]:
(start, end)

(datetime.datetime(2019, 11, 11, 0, 0),
 datetime.datetime(2022, 11, 11, 16, 11, 14, 791505))

## Download 100 assets

In [4]:
all_stocks = DataReader(stocks_list, 'yahoo', start, end)['Adj Close']

all_stocks.describe()

Symbols,ABBV,ACN,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,...,XLF,XLV,XLE,XLY,XLI,XLC,XLU,XLP,XLB,VXX
count,758.000000,758.000000,758.000000,758.000000,758.000000,758.000000,758.000000,758.000000,758.000000,758.000000,...,758.000000,758.000000,758.000000,758.000000,758.000000,758.000000,758.000000,758.000000,758.000000,758.000000
mean,107.856272,265.385165,83.909156,140.611193,113.360444,97.829768,218.609030,142.244046,188.397043,162.791588,...,31.218339,115.709650,51.477067,155.159132,87.515130,63.650455,62.693779,65.697715,71.111030,58.885548
std,26.715191,58.047625,8.124797,25.149626,14.891028,34.154505,63.170421,28.072123,33.949891,38.138545,...,5.927071,15.209944,17.521953,26.828074,13.574193,11.515599,6.384706,6.951231,12.696610,46.328515
min,56.890743,138.345749,63.700172,76.798347,76.101120,37.084091,78.421745,83.830498,110.256195,75.713028,...,16.813217,71.814056,20.812735,85.449219,46.945175,39.308067,41.590984,45.549580,36.500801,16.980000
25%,84.034914,216.911484,78.232845,122.731314,100.035469,61.111449,156.403534,118.183374,156.329662,125.167130,...,26.882692,101.910454,35.977671,136.404999,77.821486,54.294432,58.296075,60.060740,59.534719,23.370000
50%,103.460602,271.452957,82.531521,146.913567,111.983990,99.363735,237.289345,154.406998,190.812965,168.478615,...,32.392326,120.115242,48.907724,156.440292,89.881569,61.019867,62.646366,67.167805,74.005726,38.910000
75%,134.975239,311.054504,87.697998,159.075439,126.969063,131.315960,269.676071,164.609997,216.045380,198.659988,...,36.403789,129.561504,66.263403,176.383148,99.528387,75.155300,66.908028,71.172237,81.894415,75.160000
max,170.146515,409.828979,104.201859,191.601440,145.579803,165.868652,331.559998,186.570496,252.427399,223.893585,...,40.825043,141.664886,93.129997,209.854355,105.722588,85.122887,77.594452,79.452660,89.421799,276.000000


In [5]:
all_stocks

Symbols,ABBV,ACN,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,...,XLF,XLV,XLE,XLY,XLI,XLC,XLU,XLP,XLB,VXX
Date,,,,,,,,,,,,,,,,,,,,,
2019-11-11,74.547272,182.286407,80.227890,122.149216,111.417336,54.877258,149.046738,88.582497,190.269394,127.117828,...,27.952503,89.778534,50.366665,117.930771,77.746185,50.018208,55.810249,56.044235,56.492039,74.279999
2019-11-12,75.226082,183.600372,79.983871,123.597046,110.492546,54.712391,149.536087,88.900002,188.922455,127.527351,...,27.990072,90.318687,50.107937,117.852859,77.736679,50.076725,55.946659,56.228981,56.783531,73.400002
2019-11-13,75.417534,186.544800,81.213005,124.250000,112.438446,55.391216,149.056122,87.655502,191.272797,125.936737,...,27.811617,90.422920,49.890949,117.482765,77.413521,50.057220,56.774216,56.653919,56.454422,73.919998
2019-11-14,76.261688,188.223190,80.978027,124.931328,112.891205,55.236061,149.931320,87.730003,192.203918,125.584297,...,27.867971,90.432404,49.757420,117.989189,77.660637,50.281521,56.974293,56.635441,56.802338,72.720001
2019-11-15,77.131973,188.760300,80.941864,124.997551,114.750420,60.181709,150.731232,86.974503,193.930496,125.803375,...,27.943110,92.356087,50.141319,118.203468,78.230896,50.700855,57.156162,56.653919,56.755322,69.599998
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-07,148.100006,269.070007,87.181625,120.730003,107.070000,95.040001,317.959991,90.529999,162.020004,176.139999,...,34.250000,132.210007,92.989998,136.539993,96.089996,46.540001,65.809998,72.489998,76.010002,17.160000
2022-11-08,148.529999,269.029999,88.420006,123.000000,106.779999,97.459999,320.250000,89.980003,161.050003,176.089996,...,34.450001,132.869995,93.080002,136.199997,96.930000,46.590000,66.269997,72.699997,77.320000,17.660000
2022-11-09,147.619995,266.440002,87.820000,122.620003,104.370003,94.379997,315.010010,86.139999,158.979996,170.600006,...,33.869999,131.440002,88.529999,132.039993,95.580002,45.959999,65.750000,71.910004,76.070000,18.209999


In [6]:
# extract monthly returns
month_stocks = all_stocks.copy()
month_stocks['Datestring'] = month_stocks.index
    
month_stocks['year'] = pd.DatetimeIndex(month_stocks.index).year
month_stocks['month'] = pd.DatetimeIndex(month_stocks.index).month

month_on_month = month_stocks.sort_values(by='Date').groupby(['year','month']).last()
month_on_month = month_on_month.drop(month_on_month.tail(1).index).set_index(['Datestring'])

monthly_returns = month_on_month.pct_change().round(3).T
monthly_returns

Datestring,2019-11-29,2019-12-31,2020-01-31,2020-02-28,2020-03-31,2020-04-30,2020-05-29,2020-06-30,2020-07-31,2020-08-31,...,2022-01-31,2022-02-28,2022-03-31,2022-04-29,2022-05-31,2022-06-30,2022-07-29,2022-08-31,2022-09-30,2022-10-31
Symbols,,,,,,,,,,,,,,,,,,,,,
ABBV,NaN,0.009,-0.073,0.058,-0.111,0.095,0.127,0.059,-0.022,0.009,...,0.021,0.079,0.097,-0.086,0.003,0.039,-0.054,-0.063,-0.002,0.102
ACN,NaN,0.047,-0.022,-0.120,-0.096,0.139,0.089,0.065,0.051,0.067,...,-0.145,-0.106,0.067,-0.107,-0.006,-0.070,0.107,-0.058,-0.108,0.108
AEP,NaN,0.035,0.103,-0.138,-0.104,0.039,0.035,-0.066,0.091,-0.085,...,0.016,0.012,0.101,-0.007,0.038,-0.060,0.027,0.025,-0.137,0.017
AIZ,NaN,-0.013,-0.004,-0.072,-0.137,0.021,-0.028,0.007,0.040,0.137,...,-0.021,0.117,0.071,0.000,-0.025,-0.022,0.017,-0.095,-0.083,-0.065
ALLE,NaN,0.040,0.038,-0.111,-0.197,0.093,-0.008,0.028,-0.027,0.040,...,-0.073,-0.067,-0.038,0.041,-0.023,-0.120,0.080,-0.100,-0.053,0.168
AMAT,NaN,0.054,-0.050,0.005,-0.212,0.084,0.135,0.076,0.064,-0.039,...,-0.122,-0.027,-0.018,-0.163,0.065,-0.224,0.165,-0.110,-0.129,0.078
AMP,NaN,0.017,-0.007,-0.141,-0.275,0.122,0.229,0.071,0.024,0.027,...,0.009,-0.011,0.002,-0.116,0.045,-0.140,0.136,-0.002,-0.060,0.227
AMZN,NaN,0.026,0.087,-0.062,0.035,0.269,-0.013,0.130,0.147,0.090,...,-0.103,0.027,0.061,-0.238,-0.033,-0.116,0.271,-0.061,-0.109,-0.093
AVB,NaN,-0.015,0.033,-0.074,-0.259,0.107,-0.043,0.002,-0.010,0.032,...,-0.033,-0.023,0.047,-0.084,-0.086,-0.058,0.101,-0.061,-0.075,-0.049


In [7]:
# Generate monthly return forecast table
future_steps = 4
first_time = True
for ticker in stocks_list:
    print('Ticker: %s...' % ticker)
    df = monthly_returns.T[ticker].dropna().reset_index().rename(columns={'Datestring':'ds', ticker:'y'})
    model = Prophet(interval_width=0.95).fit(df)
    future_dates = model.make_future_dataframe(periods=future_steps, freq='M')
    forecast = model.predict(future_dates)
    if first_time:
        forecast_df = forecast[['ds']].set_index('ds')
        lower_df = forecast_df.copy()
        yhat_df = forecast_df.copy()
        upper_df = forecast_df.copy()
        first_time = False

    forecast = forecast.set_index('ds')

    lower_df[ticker] = forecast['yhat_lower']
    yhat_df[ticker] = forecast['yhat']
    upper_df[ticker] = forecast['yhat_upper']
        

Ticker: ABBV...


16:12:52 - cmdstanpy - INFO - Chain [1] start processing
16:12:54 - cmdstanpy - INFO - Chain [1] done processing


Ticker: ACN...


16:12:55 - cmdstanpy - INFO - Chain [1] start processing
16:12:55 - cmdstanpy - INFO - Chain [1] done processing
16:12:55 - cmdstanpy - INFO - Chain [1] start processing


Ticker: AEP...


...

16:13:53 - cmdstanpy - INFO - Chain [1] done processing
16:13:53 - cmdstanpy - INFO - Chain [1] start processing


Ticker: XLB...


16:13:53 - cmdstanpy - INFO - Chain [1] done processing
16:13:54 - cmdstanpy - INFO - Chain [1] start processing


Ticker: VXX...


16:13:54 - cmdstanpy - INFO - Chain [1] done processing


In [8]:
lower_df

,ABBV,ACN,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,...,XLF,XLV,XLE,XLY,XLI,XLC,XLU,XLP,XLB,VXX
ds,,,,,,,,,,,,,,,,,,,,,
2019-12-31,-0.031285,-0.027861,-0.030457,-0.101247,-0.051087,-0.057904,-0.101239,-0.070473,-0.095309,-0.073274,...,-0.069775,-0.004156,-0.115711,-0.063332,-0.086660,-0.027322,-0.025588,-0.017339,-0.050382,-0.440007
2020-01-31,-0.120946,-0.160965,-0.032402,-0.089058,-0.139710,-0.143498,-0.116427,-0.053492,-0.100598,-0.096085,...,-0.109496,-0.095836,-0.155615,-0.088049,-0.116987,-0.051864,-0.048366,-0.078500,-0.123254,-0.140329
2020-02-28,-0.037842,-0.161151,-0.140212,-0.066705,-0.166634,-0.046594,-0.144460,-0.061897,-0.117300,-0.144314,...,-0.116542,-0.087861,-0.205864,-0.088956,-0.115775,-0.073699,-0.115108,-0.104257,-0.099797,-0.127818
2020-03-31,-0.165864,-0.098117,-0.076716,-0.107995,-0.162357,-0.174741,-0.270321,-0.114245,-0.199498,-0.088181,...,-0.191881,-0.064303,-0.365598,-0.140211,-0.159862,-0.112159,-0.065323,-0.054664,-0.124914,0.107722
2020-04-30,-0.003224,0.024861,-0.023872,-0.017379,-0.005959,-0.043962,0.028780,0.148650,0.020726,0.059210,...,0.010744,0.043458,0.031562,0.087669,-0.015784,0.077389,-0.015893,-0.008747,0.045848,-0.531216
2020-05-29,-0.050888,-0.059725,-0.063863,-0.081826,-0.089656,-0.005668,-0.009057,-0.081952,-0.118865,-0.073656,...,-0.059312,-0.038824,-0.126614,-0.057448,-0.070350,-0.004004,-0.040964,-0.066702,-0.021658,-0.425703
2020-06-30,-0.048960,-0.032715,-0.108334,-0.102586,-0.083792,-0.034464,-0.051807,0.022176,-0.078421,-0.060762,...,-0.100314,-0.070624,-0.166111,-0.026510,-0.092321,-0.036466,-0.090717,-0.072274,-0.080545,-0.420325
2020-07-31,-0.078628,-0.045119,0.011457,-0.066662,-0.166333,-0.091347,-0.103551,-0.026677,-0.085270,-0.130560,...,-0.072166,0.002389,-0.246611,-0.042587,-0.078357,0.001075,0.007139,-0.007145,-0.023480,-0.436894
2020-08-31,-0.090566,-0.056613,-0.164327,0.005362,-0.094091,-0.234443,-0.135489,-0.018310,-0.103991,-0.103662,...,-0.097058,-0.049296,-0.280444,-0.025463,-0.100713,-0.008400,-0.095810,-0.049320,-0.091709,-0.238797


In [10]:
# Quantile table generation
timestamp = '2022-11-30'
quantile_table = pd.DataFrame([],columns=stocks_list).T
quantile_table['5'] = np.nan
quantile_table['y_hat'] = np.nan
quantile_table['95'] = np.nan

quantile_table = quantile_table.T

for ticker in stocks_list:
    q_5 = lower_df[ticker][timestamp]
    q_y = yhat_df[ticker][timestamp]
    q_95 = upper_df[ticker][timestamp]

    quantile_table[ticker] = [q_5, q_y, q_95]
    
    
quantile_table = quantile_table.T

# 95th percentile is 3 std away from mean
quantile_table['f_std'] = (quantile_table['95'] - quantile_table['y_hat']) / 3

quantile_table

,5,y_hat,95,f_std
ABBV,-0.042207,0.046844,0.133412,0.028856
ACN,-0.112527,-0.018884,0.070588,0.029824
AEP,-0.078196,-0.006893,0.076027,0.027640
AIZ,-0.071921,0.013544,0.100603,0.029020
ALLE,-0.125795,-0.001857,0.108863,0.036907
AMAT,-0.119346,-0.001103,0.111960,0.037688
AMP,-0.166665,-0.055531,0.062293,0.039275
AMZN,-0.172858,-0.089917,0.003834,0.031250
AVB,-0.075850,0.013337,0.110324,0.032329
AVY,-0.101857,-0.010958,0.078404,0.029787


In [11]:
# generate table of sampling
sampling_table = quantile_table.copy().drop(quantile_table.columns, axis=1)
for i in range(1,1001):
    sampling_table['s_%d' % i] = np.random.normal(loc=quantile_table['y_hat'], scale=quantile_table['f_std'], size=None)
    
sampling_table = sampling_table.round(4)
sampling_table

C:\Users\0074fasaet\AppData\Local\Temp\ipykernel_26176\3857847923.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sampling_table['s_%d' % i] = np.random.normal(loc=quantile_table['y_hat'], scale=quantile_table['f_std'], size=None)
C:\Users\0074fasaet\AppData\Local\Temp\ipykernel_26176\3857847923.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sampling_table['s_%d' % i] = np.random.normal(loc=quantile_table['y_hat'], scale=quantile_table['f_std'], size=None)
C:\Users\0074fasaet\AppData\Local\Temp\ipykernel_26176\385784792

C:\Users\0074fasaet\AppData\Local\Temp\ipykernel_26176\3857847923.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sampling_table['s_%d' % i] = np.random.normal(loc=quantile_table['y_hat'], scale=quantile_table['f_std'], size=None)
C:\Users\0074fasaet\AppData\Local\Temp\ipykernel_26176\3857847923.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sampling_table['s_%d' % i] = np.random.normal(loc=quantile_table['y_hat'], scale=quantile_table['f_std'], size=None)
C:\Users\0074fasaet\AppData\Local\Temp\ipykernel_26176\385784792

C:\Users\0074fasaet\AppData\Local\Temp\ipykernel_26176\3857847923.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sampling_table['s_%d' % i] = np.random.normal(loc=quantile_table['y_hat'], scale=quantile_table['f_std'], size=None)
C:\Users\0074fasaet\AppData\Local\Temp\ipykernel_26176\3857847923.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sampling_table['s_%d' % i] = np.random.normal(loc=quantile_table['y_hat'], scale=quantile_table['f_std'], size=None)
C:\Users\0074fasaet\AppData\Local\Temp\ipykernel_26176\385784792

C:\Users\0074fasaet\AppData\Local\Temp\ipykernel_26176\3857847923.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sampling_table['s_%d' % i] = np.random.normal(loc=quantile_table['y_hat'], scale=quantile_table['f_std'], size=None)
C:\Users\0074fasaet\AppData\Local\Temp\ipykernel_26176\3857847923.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sampling_table['s_%d' % i] = np.random.normal(loc=quantile_table['y_hat'], scale=quantile_table['f_std'], size=None)
C:\Users\0074fasaet\AppData\Local\Temp\ipykernel_26176\385784792

C:\Users\0074fasaet\AppData\Local\Temp\ipykernel_26176\3857847923.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sampling_table['s_%d' % i] = np.random.normal(loc=quantile_table['y_hat'], scale=quantile_table['f_std'], size=None)
C:\Users\0074fasaet\AppData\Local\Temp\ipykernel_26176\3857847923.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sampling_table['s_%d' % i] = np.random.normal(loc=quantile_table['y_hat'], scale=quantile_table['f_std'], size=None)
C:\Users\0074fasaet\AppData\Local\Temp\ipykernel_26176\385784792

...

C:\Users\0074fasaet\AppData\Local\Temp\ipykernel_26176\3857847923.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sampling_table['s_%d' % i] = np.random.normal(loc=quantile_table['y_hat'], scale=quantile_table['f_std'], size=None)
C:\Users\0074fasaet\AppData\Local\Temp\ipykernel_26176\3857847923.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sampling_table['s_%d' % i] = np.random.normal(loc=quantile_table['y_hat'], scale=quantile_table['f_std'], size=None)
C:\Users\0074fasaet\AppData\Local\Temp\ipykernel_26176\385784792

C:\Users\0074fasaet\AppData\Local\Temp\ipykernel_26176\3857847923.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sampling_table['s_%d' % i] = np.random.normal(loc=quantile_table['y_hat'], scale=quantile_table['f_std'], size=None)
C:\Users\0074fasaet\AppData\Local\Temp\ipykernel_26176\3857847923.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sampling_table['s_%d' % i] = np.random.normal(loc=quantile_table['y_hat'], scale=quantile_table['f_std'], size=None)
C:\Users\0074fasaet\AppData\Local\Temp\ipykernel_26176\385784792

C:\Users\0074fasaet\AppData\Local\Temp\ipykernel_26176\3857847923.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sampling_table['s_%d' % i] = np.random.normal(loc=quantile_table['y_hat'], scale=quantile_table['f_std'], size=None)
C:\Users\0074fasaet\AppData\Local\Temp\ipykernel_26176\3857847923.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sampling_table['s_%d' % i] = np.random.normal(loc=quantile_table['y_hat'], scale=quantile_table['f_std'], size=None)
C:\Users\0074fasaet\AppData\Local\Temp\ipykernel_26176\385784792

C:\Users\0074fasaet\AppData\Local\Temp\ipykernel_26176\3857847923.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sampling_table['s_%d' % i] = np.random.normal(loc=quantile_table['y_hat'], scale=quantile_table['f_std'], size=None)
C:\Users\0074fasaet\AppData\Local\Temp\ipykernel_26176\3857847923.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sampling_table['s_%d' % i] = np.random.normal(loc=quantile_table['y_hat'], scale=quantile_table['f_std'], size=None)
C:\Users\0074fasaet\AppData\Local\Temp\ipykernel_26176\385784792

,s_1,s_2,s_3,s_4,s_5,s_6,s_7,s_8,s_9,s_10,...,s_991,s_992,s_993,s_994,s_995,s_996,s_997,s_998,s_999,s_1000
ABBV,0.0946,0.0252,0.0425,0.0610,-0.0211,0.0477,0.0530,0.0847,0.0092,0.0514,...,0.0360,0.0345,0.0221,0.0245,0.0563,0.1013,0.0418,0.0583,0.0004,0.0644
ACN,-0.0639,-0.0172,0.0146,-0.0540,0.0013,-0.0801,-0.0253,-0.0326,-0.0225,0.0117,...,-0.0194,-0.0091,0.0281,-0.0081,-0.0277,-0.0275,-0.0159,0.0151,0.0506,0.0078
AEP,-0.0028,-0.0182,-0.0248,-0.0065,-0.0166,-0.0189,-0.0103,-0.0017,0.0120,-0.0001,...,-0.0302,0.0249,-0.0362,-0.0538,-0.0088,0.0450,-0.0140,-0.0764,-0.0542,-0.0183
AIZ,0.0499,0.0080,-0.0196,-0.0071,-0.0338,0.0297,0.0240,-0.0145,-0.0393,0.0652,...,0.0302,0.0375,-0.0151,-0.0399,-0.0333,-0.0082,-0.0144,0.0339,0.0499,0.0395
ALLE,-0.0160,0.0351,0.0055,0.0373,0.0398,-0.0438,0.0137,0.0079,0.0321,0.0303,...,-0.0394,0.0096,-0.0322,0.0658,-0.0009,-0.0481,0.0103,-0.0512,0.0158,0.0039
AMAT,0.0034,0.0102,-0.0850,0.0503,0.0098,-0.0373,0.0100,0.0070,0.0430,0.0377,...,-0.0385,0.0400,-0.0388,-0.0033,0.0190,-0.0115,0.0637,0.0033,0.0134,-0.0380
AMP,-0.1033,-0.0820,-0.0421,-0.0710,-0.0849,-0.0586,-0.0298,-0.0951,-0.0832,-0.1144,...,-0.0644,0.0293,-0.0403,0.0041,-0.0656,-0.1102,-0.0052,-0.0315,-0.0590,-0.0338
AMZN,-0.1074,-0.0457,-0.1091,-0.0830,-0.0776,-0.0734,-0.1864,-0.0513,-0.0734,-0.0452,...,-0.0696,-0.0972,-0.0902,-0.1405,-0.0989,-0.0751,-0.1104,-0.0637,-0.0797,-0.1441
AVB,0.0018,0.0388,-0.0200,0.0375,0.0450,0.0635,0.0280,0.0620,0.0085,-0.0349,...,0.0360,0.0466,0.0162,0.0161,0.0003,-0.0201,0.0443,0.0699,-0.0307,-0.0176
AVY,-0.0271,-0.0020,-0.0078,-0.0398,-0.0088,0.0280,-0.0070,0.0237,-0.0003,-0.0065,...,0.0192,-0.0002,-0.0222,0.0042,-0.0284,-0.0546,-0.0494,-0.0336,-0.0359,-0.0492


In [14]:
# extract total ranks
def generate_rank(df):
    ranks = pd.DataFrame(index=df.index, columns=df.columns)
    vector_ranks = pd.DataFrame(index=df.index, columns=df.columns)
    
    for col in df.columns:
        if df[col].isnull().all():
            continue
        ranks[col], vector_ranks[col] = generate_rank_from_values(df[col].values)
        
    return ranks, vector_ranks

def generate_rank_from_values(value_list):
    if len(value_list) != 100:
        print('Values list not 100 in number')
        return
    df = pd.DataFrame(value_list, columns=['data'])
    orig_index = df.index
    #df[241] = [random.randint(1,20) for i in range(100)]
    df = df.sort_values('data',ascending=False)
    df['my100rank'] = range(100,0,-1)
    df['my5rank'] = [5]*20 + [4]*20 + [3]*20 + [2]*20 + [1]*20
    df['pyrank'] = df['data'].rank()
    df['5rank'] = pd.qcut(df['data'], 5, labels=False, precision=1) + 1
    df['diff'] = df['my5rank'] != df['5rank']
    uniq_ranks = df[df['diff'] == True]['pyrank'].unique()
    df['adjusted5rank'] = df['my5rank']
    df['rankvector'] = np.nan
    
    df = df.join(pd.get_dummies(df['my5rank']))
    
    
    # ties on the margins of the classes
    for rank in uniq_ranks:
        rank_index = df[df['pyrank']==rank].index
        population = df['my5rank'][rank_index]
        df.loc[rank_index,'adjusted5rank'] = round(population.mean(),2)
        for p in population:
            df.loc[rank_index, p] = round(sum(population == p) / len(population),2)
            
    
    
    #print(df.to_string())
    ranks = df['adjusted5rank'][orig_index].values.tolist()
    vector_ranks = []
    for index, row in df.loc[orig_index].iterrows():
        vector_ranks.append([row[1],row[2],row[3],row[4],row[5]])
    
    # clean house
    for p in [1,2,3,4,5]:
        df[f'Rank {p}'] = df[p]
    df = df.drop(columns=[1,2,3,4,5])
    return ranks, vector_ranks

ranks, vector_ranks = generate_rank(sampling_table)
ranks

,s_1,s_2,s_3,s_4,s_5,s_6,s_7,s_8,s_9,s_10,...,s_991,s_992,s_993,s_994,s_995,s_996,s_997,s_998,s_999,s_1000
ABBV,5.0,4,5,5,3,5,5.0,5,4.0,5,...,5.0,5.0,4,4,5,5,5,5,4,5
ACN,1.0,3,4,2,4,1,2.0,2,2.5,4,...,3.0,3.0,4,3,2,2,3,4,5,4
AEP,3.0,3,3,3,3,3,3.0,3,4.0,3,...,2.0,4.0,2,1,3,5,3,1,2,3
AIZ,5.0,4,3,3,2,5,4.0,3,2.0,5,...,5.0,5.0,3,2,2,3,3,4,5,5
ALLE,3.0,5,4,5,5,2,4.0,4,4.0,5,...,2.0,4.0,2,5,4,2,4,2,4,4
AMAT,4.0,4,1,5,4,2,4.0,4,5.0,5,...,2.0,5.0,2,3,4,3,5,4,4,2
AMP,1.0,1,2,1,1,1,2.0,1,1.0,1,...,1.0,4.0,2,4,1,1,4,2,2,2
AMZN,1.0,2,1,1,1,1,1.0,2,1.0,2,...,1.0,1.0,1,1,1,1,1,2,1,1
AVB,4.0,5,3,5,5,5,5.0,5,4.0,2,...,5.0,5.0,4,4,4,2,5,5,2,3
AVY,2.0,3,3,2,3,4,3.0,4,3.0,3,...,4.0,3.0,3,4,2,2,2,2,2,2


In [16]:
pmf_vranks = []

for row in ranks.index:
    pmf, bins = np.histogram(ranks.T[row].dropna().values - 0.001, bins = range(0,6), density=True)
    pmf = np.round(pmf, 3).tolist()
    pmf_vranks.append(pmf)
    

ranks_table = ranks.copy().drop(ranks.columns, axis=1)

# rank histogram per instrument
ranks_table['PMF Rank Vector'] = pmf_vranks
# expected rank
ranks_table['PMF Rank'] = np.dot(np.array(pmf_vranks), np.array(range(1,6)))
ranks_table

,PMF Rank Vector,PMF Rank
ABBV,"[0.0, 0.008, 0.053, 0.213, 0.726]",4.657
ACN,"[0.08, 0.331, 0.309, 0.22, 0.06]",2.849
AEP,"[0.035, 0.232, 0.319, 0.328, 0.086]",3.198
AIZ,"[0.005, 0.098, 0.226, 0.393, 0.278]",3.841
ALLE,"[0.056, 0.217, 0.242, 0.289, 0.196]",3.352
AMAT,"[0.055, 0.192, 0.252, 0.28, 0.221]",3.420
AMP,"[0.439, 0.34, 0.134, 0.068, 0.019]",1.888
AMZN,"[0.84, 0.142, 0.017, 0.001, 0.0]",1.179
AVB,"[0.011, 0.097, 0.213, 0.344, 0.335]",3.895
AVY,"[0.06, 0.299, 0.29, 0.252, 0.099]",3.031


In [17]:
# break up single column vector array to separate columns
pmf_values = np.array([x for x in ranks_table['PMF Rank Vector'].values])
submission = ranks_table.filter('PMF Rank Vector')
for i in range(5):
    submission[f'Rank {i+1}'] = pmf_values[:,i]

    
diff = 1 - submission[['Rank 1','Rank 2','Rank 3','Rank 4','Rank 5']].sum(1)
submission['Rank 5'] = submission['Rank 5'] + diff

# portfolio strategy
submission['Decision'] = submission['Rank 5'] / submission['Rank 5'].sum()
# round up to 5 dp
submission = submission.round(5)

diff = 1 - submission['Decision'].sum()
submission['Decision'][-1] = submission['Decision'][-1] + diff



submission

,Rank 1,Rank 2,Rank 3,Rank 4,Rank 5,Decision
ABBV,0.000,0.008,0.053,0.213,0.726,0.03624
ACN,0.080,0.331,0.309,0.220,0.060,0.00300
AEP,0.035,0.232,0.319,0.328,0.086,0.00429
AIZ,0.005,0.098,0.226,0.393,0.278,0.01388
ALLE,0.056,0.217,0.242,0.289,0.196,0.00978
AMAT,0.055,0.192,0.252,0.280,0.221,0.01103
AMP,0.439,0.340,0.134,0.068,0.019,0.00095
AMZN,0.840,0.142,0.017,0.001,0.000,0.00000
AVB,0.011,0.097,0.213,0.344,0.335,0.01672
AVY,0.060,0.299,0.290,0.252,0.099,0.00494


In [19]:
# Save to file
submission.to_csv('submission.csv')
submission

,Rank 1,Rank 2,Rank 3,Rank 4,Rank 5,Decision
ABBV,0.000,0.008,0.053,0.213,0.726,0.03624
ACN,0.080,0.331,0.309,0.220,0.060,0.00300
AEP,0.035,0.232,0.319,0.328,0.086,0.00429
AIZ,0.005,0.098,0.226,0.393,0.278,0.01388
ALLE,0.056,0.217,0.242,0.289,0.196,0.00978
AMAT,0.055,0.192,0.252,0.280,0.221,0.01103
AMP,0.439,0.340,0.134,0.068,0.019,0.00095
AMZN,0.840,0.142,0.017,0.001,0.000,0.00000
AVB,0.011,0.097,0.213,0.344,0.335,0.01672
AVY,0.060,0.299,0.290,0.252,0.099,0.00494
